In [1]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import norm

# Set plot style
sns.set_style('darkgrid')

from src.stock_path_simulator import StockPathSimulator

# Finite american options 

Finite american put options don't have an analytic equation. While the continuation equation for American put option is the European put option, there's a boundary for optimal time for expiration.

# Differential equations


The value is given by the differential equation for perpetual American option

$$
r v_{L^*}(x) - r x v'_{L^*}(x)-\frac{1}{2}\sigma^2 x^2 v''_{L^*}(x)=rK (0<x\leq L^*<k, \text{otherwise} 0)
$$

which is the (time-independent) Black-Scholes equation and originates from the linear complementary conditions

$$
\begin{align}
v(x) &\geq (K-x)^+ \text{for all} x\geq 0
r v(x) - r x v'(x)-\frac{1}{2}\sigma^2 x^2 v''(x) &\geq 0 \text{for all} x\geq 0
\end{align}
$$

and for each $x\geq 0$ equality holds in either of them.

For finite time put options, instead you get the standard Black-Scholes PDE


$$
r v(t,x) -  v_t(t,x)-r x v_{x}(t, xx)-\frac{1}{2}\sigma^2 x^2 v_{xx}(t,x)=rK (0<x\leq \{(t,x); v(t,x)>(K-x)^+\})
$$

# Simulation

In practice, finite-expiration American options can be simulated with methods like binomial tree and Longstaff-Schwarz algorithm, with packages that implement these. 

# Binomial tree simulation

Here we implement a Cox-Ross-Rubinstein logic. This has a binomial tree with an up step and downstep where the asset price $S\rightarrow uS$ or $S\rightarrow dS$ depending on which step, with probability of $p$ and $1-p$ respectively. We pick the probabliity such that this replicates the mean and variance of a geometric Brownian motion. IN CRR, this sets $u=e^{\sigma \sqrt{\Delta t}}$, $d=e^{\sigma \sqrt{-\Delta t}}$, $p = \frac{e^{r\Delta t}- d}{u-d}$ 

In [2]:
class BinomialTreeSimulator:
    def __init__(self, S0, K, T, r, sigma, n_steps, option_type='put'):
        self.S0 = S0  # Initial stock price
        self.K = K    # Strike price
        self.T = T    # Time to maturity
        self.r = r    # Risk-free rate
        self.sigma = sigma  # Volatility
        self.n_steps = n_steps  # Number of steps in the binomial tree
        self.option_type = option_type.lower()  # 'put' or 'call'
        self.dt = T / n_steps  # Time increment
        self.u = np.exp(sigma * np.sqrt(self.dt))  # Up factor from Cox-Ross-Rubinstein model
        self.d = 1 / self.u  # Down factor
        self.p = (np.exp(r * self.dt) - self.d) / (self.u - self.d)  # Risk-neutral probability
        self.asset_prices = self.create_binomial_tree() # Create the binomial tree
        self.option_prices = self.create_european_option_prices() # Assign option prices for each node in binomial tree

    def create_binomial_tree(self):
        # This creates an array storing asset prices at each node
        # This array is triangular in shape—at each timestep i, there are i+1 nodes
        # Here, we use a 2D array with padding for simplicity
        asset_prices = np.zeros((self.n_steps + 1, self.n_steps + 1))
        for i in range(self.n_steps + 1):
            for j in range(i + 1):
                asset_prices[i, j] = self.S0 * (self.u ** (i - j)) * (self.d ** j)
        return asset_prices
    
    def calculate_payoff(self, stock_price):
        """Calculate option payoff at expiration"""
        if self.option_type == 'put':
            return max(self.K - stock_price, 0)
        elif self.option_type == 'call':
            return max(stock_price - self.K, 0)
        else:
            raise ValueError("option_type must be 'put' or 'call'")

    def create_european_option_prices(self):
        # This computes the European option price at each node using backward induction
        option_prices = np.zeros((self.n_steps + 1, self.n_steps + 1))
        
        # Compute option values at maturity
        for j in range(self.n_steps + 1):
            option_prices[self.n_steps, j] = self.calculate_payoff(self.asset_prices[self.n_steps, j])
        
        # Backward induction to calculate option prices at earlier nodes
        for i in range(self.n_steps - 1, -1, -1):
            for j in range(i + 1):
                expected_value = (self.p * option_prices[i + 1, j] + (1 - self.p) * option_prices[i + 1, j + 1]) * np.exp(-self.r * self.dt)
                option_prices[i, j] = expected_value

        return option_prices
    
    def price_european_option(self):    
        # Price a European option using the binomial tree
        european_option_price = self.option_prices[0, 0]
        return european_option_price
        
    def price_american_option(self):
        # Price the American option using the binomial tree with early exercise feature
        # We examine at each node whether the option should be exercised or held
        american_option_prices = np.zeros((self.n_steps + 1, self.n_steps + 1))
        
        # First, set the terminal condition (values at maturity) - same as European
        for j in range(self.n_steps + 1):
            american_option_prices[self.n_steps, j] = self.calculate_payoff(self.asset_prices[self.n_steps, j])
        
        # Backward induction to calculate option prices at earlier nodes
        for i in range(self.n_steps - 1, -1, -1):
            for j in range(i + 1):
                # Calculate continuation value using the American option prices we're computing
                continuation_value = (self.p * american_option_prices[i + 1, j] + 
                                    (1 - self.p) * american_option_prices[i + 1, j + 1]) * np.exp(-self.r * self.dt)
                exercise_value = self.calculate_payoff(self.asset_prices[i, j])
                american_option_prices[i, j] = max(continuation_value, exercise_value)
        
        return american_option_prices[0, 0]

In [3]:
# Black-Scholes formulas for both puts and calls
def black_scholes_option_price(S0, K, T, r, sigma, option_type='put'):
    """Calculate Black-Scholes option price for puts and calls"""
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type.lower() == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    elif option_type.lower() == 'call':
        option_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        raise ValueError("option_type must be 'put' or 'call'")
    
    return option_price

In [10]:
S0 = 140       # Initial stock price
K = 140        # Strike price
sigma = 0.3    # Volatility
r = 0.035      # Risk-free rate
T = 10         # Time horizon (years)
mu = 0         # Drift (risk-neutral)
n_steps = 1000  # Number of steps in binomial tree

print("=== PUT OPTION COMPARISON ===")
# PUT option
put_binomial_tree = BinomialTreeSimulator(S0, K, T, r, sigma, n_steps=n_steps, option_type='put')

european_put_price_binomial = put_binomial_tree.price_european_option()
print(f"European Put Option Price from Binomial Tree: {european_put_price_binomial:.4f}")

european_put_price_black_scholes = black_scholes_option_price(S0, K, T, r, sigma, 'put')
print(f"European Put Option Price from Black-Scholes: {european_put_price_black_scholes:.4f}")
print(f"Put Price Difference: {abs(european_put_price_binomial - european_put_price_black_scholes):.6f}")

print("\n=== CALL OPTION COMPARISON ===")
# CALL option
call_binomial_tree = BinomialTreeSimulator(S0, K, T, r, sigma, n_steps=n_steps, option_type='call')

european_call_price_binomial = call_binomial_tree.price_european_option()
print(f"European Call Option Price from Binomial Tree: {european_call_price_binomial:.4f}")

european_call_price_black_scholes = black_scholes_option_price(S0, K, T, r, sigma, 'call')
print(f"European Call Option Price from Black-Scholes: {european_call_price_black_scholes:.4f}")
print(f"Call Price Difference: {abs(european_call_price_binomial - european_call_price_black_scholes):.6f}")

# Verify Put-Call Parity: C - P = S0 - K*e^(-rT)
put_call_parity_lhs = european_call_price_black_scholes - european_put_price_black_scholes
put_call_parity_rhs = S0 - K * np.exp(-r * T)
print(f"\n=== PUT-CALL PARITY CHECK ===")
print(f"C - P = {put_call_parity_lhs:.4f}")
print(f"S0 - K*e^(-rT) = {put_call_parity_rhs:.4f}")
print(f"Put-Call Parity Difference: {abs(put_call_parity_lhs - put_call_parity_rhs):.6f}")

=== PUT OPTION COMPARISON ===
European Put Option Price from Binomial Tree: 25.5227
European Put Option Price from Black-Scholes: 25.5339
Put Price Difference: 0.011241

=== CALL OPTION COMPARISON ===
European Call Option Price from Binomial Tree: 66.8664
European Call Option Price from Black-Scholes: 66.8776
Call Price Difference: 0.011241

=== PUT-CALL PARITY CHECK ===
C - P = 41.3437
S0 - K*e^(-rT) = 41.3437
Put-Call Parity Difference: 0.000000


In [13]:
# Compare American vs European option prices for both puts and calls

print("=== AMERICAN PUT OPTION ===")
american_put_price_binomial = put_binomial_tree.price_american_option()
print(f"American Put Option Price from Binomial Tree: {american_put_price_binomial:.4f}")
print(f"European Put Option Price from Binomial Tree: {european_put_price_binomial:.4f}")

# Premium from early exercise for puts
put_early_exercise_premium = american_put_price_binomial - european_put_price_binomial
print(f"Put Early Exercise Premium: {put_early_exercise_premium:.4f}")

print("\n=== AMERICAN CALL OPTION ===")
american_call_price_binomial = call_binomial_tree.price_american_option()
print(f"American Call Option Price from Binomial Tree: {american_call_price_binomial:.4f}")
print(f"European Call Option Price from Binomial Tree: {european_call_price_binomial:.4f}")

# Premium from early exercise for calls
call_early_exercise_premium = american_call_price_binomial - european_call_price_binomial
print(f"Call Early Exercise Premium: {call_early_exercise_premium:.4f} (should be zero)")

=== AMERICAN PUT OPTION ===
American Put Option Price from Binomial Tree: 32.8105
European Put Option Price from Binomial Tree: 25.5227
Put Early Exercise Premium: 7.2877

=== AMERICAN CALL OPTION ===
American Call Option Price from Binomial Tree: 66.8664
European Call Option Price from Binomial Tree: 66.8664
Call Early Exercise Premium: 0.0000 (should be zero)
